In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
# !curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
# conda_installer.install()
conda_installer.install()
# !/root/miniconda/bin/conda info -e
import rdkit

from deepchem.feat.graph_features import atom_features as get_atom_features

2022-12-20 08:47:34.954987: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-20 08:47:35.076821: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-20 08:47:35.076843: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-20 08:47:35.603903: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
sample_structure = rdkit.Chem.rdmolfiles.MolFromPDBFile('../Datasets/pdb-test/1a1e.pdb')

In [3]:
df = pd.read_csv('../Datasets/pdbbind.csv')

In [4]:
cols = [x for x in df.columns if (x.__contains__('gb-') or x.__contains__('vdwaals')) and not x.__contains__('etot')]

In [5]:
sample_info = df[df['complex-name'] == '1a1e'][cols].to_numpy()
sample_info = np.array(sample_info[0])

In [6]:
sample_info

array([ 7.03070120e+03, -1.35150036e+04, -3.34937910e+03,  5.65130000e+01,
        7.18952230e+03, -1.33669577e+04, -3.37817030e+03,  5.62103000e+01,
       -1.58821000e+02, -9.66715000e+01, -6.91305000e+01,  4.45980000e+00,
       -1.95158170e+03, -1.90104860e+03, -1.14008000e+01])

In [7]:
def convertBONDtoOneHotVector(bond_type):
    BOND_DICT = {dir(rdkit.Chem.rdchem.BondType)[i]: i for i in range(22)}
    one_hot = np.zeros(22)
    one_hot[BOND_DICT[str(bond_type)]] = 1
    return one_hot

In [8]:
atom_features = []
adjacency_list = []
for atom in sample_structure.GetAtoms():
    new_feature = get_atom_features(atom).tolist()
    position = sample_structure.GetConformer().GetAtomPosition(atom.GetIdx())
    new_feature += [atom.GetMass(), atom.GetAtomicNum()]
    new_feature += [position.x, position.y, position.z]
    atom_features.append(new_feature)
atom_features = np.array(atom_features)

In [9]:
for atom in sample_structure.GetAtoms():
    atom_bonds = []
    for neighbor in atom.GetNeighbors():
        bond_type = sample_structure.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx()).GetBondType()
        neighbor_idx = neighbor.GetIdx()
        atom_bonds.append([neighbor_idx, convertBONDtoOneHotVector(bond_type)])
    adjacency_list.append(atom_bonds)

In [10]:
%cd ..

/home/ali/PycharmProjects/GBNN


In [11]:
from PGGCN.layers import RuleGraphConvLayer, ConvLayer

In [13]:
class ConvLayer(tf.keras.Model):
    def __init__(self, out_channel, num_features=40):
        super(ConvLayer, self).__init__()
        self.out_channel = out_channel
        self.num_features = num_features
        self.w = tf.Variable(tf.initializers.glorot_uniform()
                             (shape=[num_features, out_channel]), shape=[num_features, out_channel], trainable=True)

    def _call_single(self, inp):
        features = inp[0]
        out = [0] * self.out_channel
        for feature in features:
            feature = tf.reshape(feature, [1, -1])
            out += tf.nn.softmax(tf.matmul(feature, self.w))
        return tf.reshape(out, [-1])

    def call(self, inputs):
        output = []
        for inp in inputs:
            output.append(self._call_single(inp))
        return tf.reshape(output, [len(inputs), -1])

In [26]:
class PGGCN(tf.keras.Model):
    def __init__(self, num_atom_features = 80, r_out_channel = 40, c_out_channel = 1024):
        super().__init__()
        self.ruleGraphConvLayer = RuleGraphConvLayer(r_out_channel, num_atom_features)
        self.conv = ConvLayer(c_out_channel)
        self.dense1 = tf.keras.layers.Dense(650, activation='sigmoid')
        self.dense2 = tf.keras.layers.Dense(1, activation='relu')
        self.dense3 = tf.keras.layers.Dense(1,
                 kernel_initializer=tf.keras.initializers.Constant([.5, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1]),
                 bias_initializer=tf.keras.initializers.Zeros(), activation=tf.keras.activations.relu)

    def set_physics_info(self, info):
        self.physics_info = info.reshape([-1, 15])

    def addRule(self, rule, start_index, end_index = None):
        self.ruleGraphConvLayer.addRule(rule, start_index, end_index)

    def call(self, inputs):
        x = self.ruleGraphConvLayer(inputs)
        x = self.conv(x)
        x = self.dense1(x)
        model_var = self.dense2(x)
        print("model var shape ", model_var.shape)
        merged = tf.concat([model_var, self.physics_info], axis=1)
        print("merged shape", merged.shape)
        out = self.dense3(merged)
        return out

m = PGGCN()
m.addRule("sum", 0, 75)
m.addRule("multiply", 75, 77)
m.addRule("distance", 77, 80)
m.set_physics_info(sample_info)

In [27]:
out = m([[atom_features, adjacency_list]])

model var shape  (1, 1)
merged shape (1, 16)


,complex-name,gb-complex-etot,gb-complex-1-4-eel,gb-complex-eelec,gb-complex-egb,gb-complex-esurf,gb-protein-etot,gb-protein-1-4-eel,gb-protein-eelect,gb-protein-egb,...,pb-protein-epb,pb-protein-ecavity,pb-ligand-etot,pb-ligand-vdwaals,pb-ligand-eelec,pb-ligand-epb,pb-ligand-ecavity,ddg,enthalpy-gb,entropy
